In [20]:
import pandas as pd
import matplotlib.pyplot as plt

import pyarrow.parquet as pq
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


data = pd.read_csv('/Users/Boaz/Documents/DataMining/prepro_data_RF.csv')
data = data.drop('time',axis = 1)
data = data.drop('n_days_id',axis = 1)
data = data.drop('season',axis =1 )
data = data.drop('id', axis = 1)



# Variable: circumplex.arousal   Importance: 0.02
# Variable: appCat.finance       Importance: 0.02
# Variable: appCat.game          Importance: 0.02
# Variable: appCat.unknown       Importance: 0.02
# Variable: appCat.weather       Importance: 0.01
# Variable: summer               Importance: 0.01
# Variable: spring               Importance: 0.0

# data = data.drop('circumplex.arousal',axis = 1)
# data = data.drop('appCat.finance',axis = 1)
# data = data.drop('appCat.game',axis =1 )
# data = data.drop('appCat.unknown', axis = 1)
# data = data.drop('appCat.weather',axis = 1)
# data = data.drop('summer',axis =1 )
# data = data.drop('spring', axis = 1)

In [21]:
from collections import Counter
count = Counter(data['target_mood_plus1'])
count



Counter({6.0: 248, 7.0: 578, 8.0: 288, 4.0: 8, 5.0: 21, 9.0: 9, 3.0: 2})

In [22]:
data = data[data.target_mood_plus1 != 3]
count = Counter(data['target_mood_plus1'])
count

Counter({6.0: 248, 7.0: 578, 8.0: 288, 4.0: 8, 5.0: 21, 9.0: 9})

In [23]:
from sklearn.model_selection import train_test_split
X_train_total = []
X_test_total = []
Y_train_total = []
Y_test_total = []
labels = np.array(data['target_mood_plus1'])
features= data.drop('target_mood_plus1', axis = 1)
features = np.array(features)
Xtrain, Xtest, ytrain, ytest = train_test_split(features, labels, random_state = 42,stratify=labels)
X_train_total.append(Xtrain)
X_test_total.append(Xtest)
Y_train_total.append(ytrain)
Y_test_total.append(ytest)


In [24]:
# GET RID OF NESTED LIST
X_train_total = [item for l in X_train_total for item in l]
X_test_total = [item for l in X_test_total for item in l]
Y_train_total = [item for l in Y_train_total for item in l]
Y_test_total = [item for l in Y_test_total for item in l]

In [25]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [9]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train_total, Y_train_total)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   60.0s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.4min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=100,
                              

In [26]:
rf_random.best_params_

{'n_estimators': 200,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 50,
 'bootstrap': True}

In [27]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'bootstrap': [True],
    'max_depth': [45, 50, 55],
    'max_features': ['sqrt'],
    'min_samples_leaf': [1,2, 3],
    'min_samples_split': [2, 1, 3],
    'n_estimators': [150,200, 250]
}
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [28]:
grid_search.fit(X_train_total, Y_train_total)

Fitting 3 folds for each of 81 candidates, totalling 243 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:    8.9s
[Parallel(n_jobs=-1)]: Done 243 out of 243 | elapsed:   13.3s finished


GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jo

In [29]:
grid_search.best_params_

{'bootstrap': True,
 'max_depth': 45,
 'max_features': 'sqrt',
 'min_samples_leaf': 3,
 'min_samples_split': 2,
 'n_estimators': 150}

In [37]:
best_grid = grid_search.best_estimator_

best_grid.fit(X_train_total, Y_train_total)
predictions = best_grid.predict(X_test_total)
# predictions = [np.argmax(p) for p in predictions]
#predictions

In [38]:
predictions = predictions.round()
print(accuracy_score(Y_test_total, predictions))
print(classification_report(Y_test_total, predictions))
from sklearn.metrics import mean_squared_error, mean_absolute_error
print(mean_squared_error(Y_test_total, predictions))

0.5138888888888888
              precision    recall  f1-score   support

         4.0       0.00      0.00      0.00         2
         5.0       0.00      0.00      0.00         5
         6.0       0.46      0.18      0.26        62
         7.0       0.52      0.84      0.64       145
         8.0       0.52      0.21      0.30        72
         9.0       0.00      0.00      0.00         2

    accuracy                           0.51       288
   macro avg       0.25      0.20      0.20       288
weighted avg       0.49      0.51      0.45       288

0.6076388888888888


C:\Users\Boaz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [39]:
# Get numerical feature importances
features = data.drop('target_mood_plus1', axis = 1)
feature_list = list(features.columns)

importances = list(best_grid.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: mood                 Importance: 0.19
Variable: activity             Importance: 0.1
Variable: appCat.other         Importance: 0.1
Variable: screen               Importance: 0.09
Variable: appCat.communication Importance: 0.09
Variable: appCat.entertainment Importance: 0.09
Variable: appCat.builtin       Importance: 0.08
Variable: appCat.social        Importance: 0.08
Variable: appCat.office        Importance: 0.04
Variable: appCat.travel        Importance: 0.04
Variable: appCat.utilities     Importance: 0.04
Variable: circumplex.valence   Importance: 0.03
Variable: sms                  Importance: 0.03


In [ ]:
best_grid = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mae',
                      max_depth=90, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=2,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=1500, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

best_grid.fit(X_train_total, Y_train_total)
predictions = best_grid.predict(X_test_total)
# predictions = [np.argmax(p) for p in predictions]
predictions = predictions.round()
print(accuracy_score(Y_test_total, predictions))
print(classification_report(Y_test_total, predictions))
from sklearn.metrics import mean_squared_error, mean_absolute_error
print(mean_absolute_error(Y_test_total, predictions))
